## Prediccion de Compra de Plaza de Liga Nacional Temporada 2025/2026 con Machine Learning

En este estudio vamos a predecir con algoritmos de Machine Learning cuales son los maximos candidatos a reemplazar a Argentino de Junin y Riachuelo de La Rioja en la temporada 25/26 de la Liga Nacional .


**ATENCION**: Este estudio solo tiene la intencion de entretenimiento y curiosidad, las predicciones aqui no implican que si o si el equipo predicho sea el que ocupe la plaza de los equipos que se bajan. Ademas al ser modelos de prediccion en base a datos recolectados, este tampoco representa la realidad actual de los clubes, solo intenta buscar una tendencia. Desde ya, gracias

### Importacion de Datos

Aqui dividiremos el trabajo en dos bases de datos (o data frames), una de **entrenamiento**, que seran los datos de todos los equipos en las temporadas donde hubo invitaciones o compras de plaza a Liga Nacional (desde la temporada 2004/05 a la 2020/21, no todas las temporadas hubo compras de plaza).

A esa base de datos de entrenamiento lo llamaremos dataTrain y posee las siguientes columnas:
- Club (nombre del club)
- Capacidad (capacidad del estadio donde suele hacer de local)
- Prioridad (la posicion en la tabla dentro de los no ascendidos)
- Tamano_Ciudad (tamaño de la ciudad donde esta el club)
- Compro_Plaza (si compro plaza esa temporada)
- Conferencia (conferencia donde jugo)
- Solvencia_Economica (si puede pagar o no plaza o tuvo problemas economicos en las ultimas 3 temporadas)

Algunas aclaraciones:
- Tamaño de Ciudad: Nos referimos a la cantidad de poblacion de la ciudad donde esta el club. Sin embargo si este club esta en una ciudad conurbanada, se tomara la poblacion de la ciudad principal del respectivo conurbano. Por ejemplo el caso de Olimpico de La Banda, esta en la ciudad de La Banda (Santiago del Estero), pero se tomara la poblacion de Santiago del Estero Capital, lo mismo pasaria con Lanus (de Lanus, pero tomaremos la data de Capital Federal). Las unicas excepciones a la regla son conurbanos que estan en mas de una provincia, caso Carmen de Patagones-Viedma o Cipolletti-Neuquen que las tomaremos como ciudades distintas
- Prioridad: Como no todas las ligas tuvieron la misma cantidad de equipos, tomaremos como base la cantidad de equipos de esta temporada y con las ligas anteriores, haremos un ranking equivalente. Por ejemplo un equipo que salio ultimo en el puesto 14 (de 14), en la prioridad figurara 31, ya que es la cantidad de equipos de la temporada pasada que no ascendieron.
- Conferencia: Los valores son 0 para la Conferencia Norte y 1 para la Conferencia Sur
- Compro_Plaza: Los valores son 0 si no compro plaza, y 1 si compro plaza
- Solvencia_Economica: Los valores son 0 si no es solvente economicamente el equipo, 1 si es solvente

In [206]:
import pandas as pd
dataTrain=pd.read_csv("CompraPlaza.csv",sep=";")
dataTrain

,Club,Capacidad,Prioridad,Tamano_Ciudad,Compro_Plaza,Conferencia,Solvencia_Economica
0,Ameghino,500,7,79351,0,0,0
1,Atenas,2000,12,20533,0,1,0
2,Barrio Parque,1000,10,1317298,0,0,1
3,Central Argentino Olímpico,1500,27,14499,0,0,0
4,Ciclista Juninense,3100,14,87509,0,1,1
...,...,...,...,...,...,...,...
135,Independiente,1000,22,231198,0,1,1
136,Alma Juniors,1000,24,40125,0,0,1
137,Ferro Carril Oeste,4500,27,2936877,0,1,0
138,San Martín,1000,29,87509,0,1,1


Ahora toca importar la otra base de datos, la de **test**, que seran los datos de todos los equipos en la temporada pasada que no ascendieron.

A esa base de datos de entrenamiento lo llamaremos dataTest y posee las siguientes columnas:
- Club (nombre del club)
- Capacidad (capacidad del estadio donde suele hacer de local)
- Prioridad (la posicion en la tabla dentro de los no ascendidos)
- Tamano_Ciudad (tamaño de la ciudad donde esta el club)
- Conferencia (conferencia donde jugo)

In [207]:

dataTest=pd.read_csv("Equipos2425.csv",sep=";")
dataTest

,Club,Capacidad,Prioridad,Tamano_Ciudad,Conferencia,Solvencia_Economica
0,Barrio Parque,700,16,1317298,0,1
1,Centenario,1000,10,75437,0,1
2,Colón,400,28,391164,0,0
3,Deportivo Norte,1200,18,11181,0,1
4,Provincial,10000,26,1193605,0,0
5,Rivadavia,900,20,82582,0,0
6,San Isidro,2200,1,61750,0,0
7,Sportivo Suardi,1200,4,5165,0,0
8,Amancay,1200,5,178872,0,0
9,Comunicaciones,3500,11,33551,0,0


Para simular usaremos **Random Forest** un muy buen algoritmo basado en arboles de decision, que van simulando "en base a preguntas binarias, es decir si o no" y de ahi hacen varias predicciones. Aparte es rapido, eficiente y pondera mucho variables claves (cosa que otros algoritmos como KNN no lo hacen)

Importamos Librerias ideales para el modelo y buscamos los mejores parametros para predecir

In [208]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import  confusion_matrix
from sklearn.metrics import  roc_auc_score,accuracy_score

from sklearn.ensemble import RandomForestClassifier

In [209]:
rf= RandomForestClassifier(random_state=13)
rf_params=[{"n_estimators":[50,100,200,500]},{"criterion":["gini", "entropy", "log_loss"]},{"class_weight":["balanced", "balanced_subsample"]},
 {"max_depth":[None,10,20,30,40]},{"min_samples_split":[2,5,10]},{"min_samples_leaf":[1,2,4]},{"max_features":["sqrt","log2"]}]
rf_cv=RandomizedSearchCV(rf,param_distributions=rf_params,n_iter=5,random_state=13,scoring="roc_auc",n_jobs=-1)

Ya con el algoritmo listo, lo entrenamos con los datos de entrenamiento (dataTrain)

In [210]:
rf_cv.fit(dataTrain.drop(["Club","Compro_Plaza"],axis=1),dataTrain["Compro_Plaza"])

RandomizedSearchCV(estimator=RandomForestClassifier(random_state=13), n_iter=5,
                   n_jobs=-1,
                   param_distributions=[{'n_estimators': [50, 100, 200, 500]},
                                        {'criterion': ['gini', 'entropy',
                                                       'log_loss']},
                                        {'class_weight': ['balanced',
                                                          'balanced_subsample']},
                                        {'max_depth': [None, 10, 20, 30, 40]},
                                        {'min_samples_split': [2, 5, 10]},
                                        {'min_samples_leaf': [1, 2, 4]},
                                        {'max_features': ['sqrt', 'log2']}],
                   random_state=13, scoring='roc_auc')

Con el modelo ya entrenado y con los mejores (o mas exactos) parametros, ya es hora de predecir

In [211]:
rf_esti= rf_cv.best_estimator_

En algoritmos de clasificacion, si una variable pertenece a 0 o a 1, se calcula en base a prioridades, normalmente si es menor o igual a 0.5 va 0, de lo contrario 1. En este caso calcularemos las probabilidad, para manejar a mano ese umbral donde decretamos los 1s o 0s. En este caso le pondremos 0.1 (mayor a  10% va 1).

In [212]:
probs = rf_esti.predict_proba(dataTest.drop("Club", axis=1))[:,1]
preds_rf = (probs >= 0.1).astype(int)

Con las predicciones llenamos los valores de Compro_Plaza en nuestra base de datos de test (la actual temporada) 

In [213]:
dataTest["Compro_Plaza"]=preds_rf


In [214]:
dataTest

,Club,Capacidad,Prioridad,Tamano_Ciudad,Conferencia,Solvencia_Economica,Compro_Plaza
0,Barrio Parque,700,16,1317298,0,1,0
1,Centenario,1000,10,75437,0,1,0
2,Colón,400,28,391164,0,0,0
3,Deportivo Norte,1200,18,11181,0,1,0
4,Provincial,10000,26,1193605,0,0,0
5,Rivadavia,900,20,82582,0,0,0
6,San Isidro,2200,1,61750,0,0,1
7,Sportivo Suardi,1200,4,5165,0,0,0
8,Amancay,1200,5,178872,0,0,1
9,Comunicaciones,3500,11,33551,0,0,0


Filtraremos los clubes solventes y posibles compradores

In [215]:
ClubesSolventesCompradores=dataTest[(dataTest["Solvencia_Economica"]==1)&(dataTest["Compro_Plaza"]==1)]

In [216]:
ClubesSolventesCompradores

,Club,Capacidad,Prioridad,Tamano_Ciudad,Conferencia,Solvencia_Economica,Compro_Plaza
15,El Talar,6000,17,2936877,1,1,1
19,Lanús,3000,2,2936877,1,1,1
25,Pico FC,2000,8,56795,1,1,1
26,Quilmes,8000,14,593337,1,1,1
27,Union,8000,19,593337,1,1,1


Vemos que los clubes compradores serian:
- El Talar
- Lanus (que compraria plaza o seria invitado en lugar de Riachuelo)
- Pico Football Club
- Quilmes MDP
- Union MDP

<img src="logoTalar.png"  width="200"> <img src="LogoLanus.png" width="200"> <img src="logoPico.jpg" width="200"> <img src="LogoQuilmes.jpg" width="200"> <img src="logoUnion.png" width="200">  

### Y si los equipos usan su prioridad real? (es decir antes si vos era 8vo eras de los peorcito, ahora sos de los mejores)

Eso solucionaremos ahora, hacemos lo mismo que el anterior, solo que habra mientras el dataTest sera el mismo, dataTrain usara su posicion real, 
mas no equivalente. A ambos datasets llamaremos dataTestB y dataTrainB respectivamente

Importamos Datos

In [217]:
dataTrainB=pd.read_csv("CompraPlazaA.csv",sep=";")
dataTestB=pd.read_csv("Equipos2425.csv",sep=";")

Entrenamos de nuevo

In [218]:
rf= RandomForestClassifier(random_state=13)
rf_params=[{"n_estimators":[50,100,200,500]},{"criterion":["gini", "entropy", "log_loss"]},{"class_weight":["balanced", "balanced_subsample"]},
 {"max_depth":[None,10,20,30,40]},{"min_samples_split":[2,5,10]},{"min_samples_leaf":[1,2,4]},{"max_features":["sqrt","log2"]}]
rf_cv=RandomizedSearchCV(rf,param_distributions=rf_params,n_iter=5,random_state=13,scoring="roc_auc",n_jobs=-1)
rf_cv.fit(dataTrainB.drop(["Club","Compro_Plaza"],axis=1),dataTrainB["Compro_Plaza"])
rf_esti= rf_cv.best_estimator_

Predecimos

In [220]:
probs = rf_esti.predict_proba(dataTestB.drop("Club", axis=1))[:,1]
preds_rf = (probs >= 0.1).astype(int)
dataTestB["Compro_Plaza"]=preds_rf


Buscamos clubes candidatos

In [221]:
ClubesSolventesCompradoresB=dataTest[(dataTestB["Solvencia_Economica"]==1)&(dataTestB["Compro_Plaza"]==1)]
ClubesSolventesCompradoresB

,Club,Capacidad,Prioridad,Tamano_Ciudad,Conferencia,Solvencia_Economica,Compro_Plaza
15,El Talar,6000,17,2936877,1,1,1
19,Lanús,3000,2,2936877,1,1,1
26,Quilmes,8000,14,593337,1,1,1
27,Union,8000,19,593337,1,1,1


Y aca vemos que son los mismos equipos a excepcion de Pico Football.

Por lo que concluimos que el equipo que posiblemente reemplace a Argentino de Junin podrian ser:
- El Talar (campeon liga femenina, cancha grande en Argentinos Juniors, campeon del pasado Federal)
- Quilmes (si bien Union tambien esta, pero lo ayudo mucho el dato que hace de local en el estadio Islas Malvinas de 8000 espectadores, 
a nivel historico, dirigencial, estructura deportiva, me inclino por los marplatenses

<img src="logoTalar.png"  width="300"> <img src="LogoQuilmes.jpg" width="300"> 